In [1]:
# Chattea Intent Classifier - MLP + Sentence Transformers
# Complete Working Pipeline in Notebook Format
# Ready to run cell by cell!

"""
JUPYTER NOTEBOOK STRUCTURE:
Run each cell in order (Shift+Enter)

Required Files:
- chattea_dataset.csv (text, intent columns)
- responses.json (your bilingual responses)

Installation:
!pip install torch sentence-transformers scikit-learn pandas difflib
"""

# ============================================================================
# CELL 1: IMPORTS AND SETUP
# ============================================================================

import json
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer, util
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from difflib import get_close_matches
import warnings
warnings.filterwarnings('ignore')

print("=" * 80)
print("CHATTEA INTENT CLASSIFIER - MLP + SENTENCE TRANSFORMERS")
print("=" * 80)
print(f"PyTorch Version: {torch.__version__}")
print("=" * 80)

c:\Users\nicho\Documents\GitHub_Repository\Python\chattea-ai\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CHATTEA INTENT CLASSIFIER - MLP + SENTENCE TRANSFORMERS
PyTorch Version: 2.9.1+cu130


In [2]:
# ============================================================================
# CELL 2: DEVICE CONFIGURATION
# ============================================================================

# Set device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️  Using device: {device}")

if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️  No GPU detected - training will use CPU (slower)")


🖥️  Using device: cuda
🎮 GPU: NVIDIA GeForce RTX 4060 Laptop GPU
💾 GPU Memory: 8.59 GB


In [3]:
# ============================================================================
# CELL 3: LOAD DATA
# ============================================================================

print("\n" + "=" * 80)
print("📂 LOADING DATA")
print("=" * 80)

# Load training dataset
df = pd.read_csv("chattea_dataset.csv")
print(f"✓ Loaded dataset: {len(df)} samples")
print(f"✓ Columns: {list(df.columns)}")
print(f"✓ Unique intents: {df['intent'].nunique()}")

# Show sample data
print("\n📊 Sample data:")
print(df.head(10))

# Intent distribution
print("\n📈 Intent distribution:")
intent_counts = df['intent'].value_counts()
print(intent_counts.head(15))

# Load responses
with open("responses.json", "r", encoding="utf-8") as f:
    RESPONSES = json.load(f)
print(f"\n✓ Loaded responses for {len(RESPONSES)} intents")


📂 LOADING DATA
✓ Loaded dataset: 657 samples
✓ Columns: ['text', 'intent']
✓ Unique intents: 33

📊 Sample data:
           text    intent
0         hello  greeting
1            hi  greeting
2     hey there  greeting
3  good morning  greeting
4  good evening  greeting
5      hi there  greeting
6           hey  greeting
7     greetings  greeting
8     what's up  greeting
9     hello bot  greeting

📈 Intent distribution:
intent
greeting            20
gratitude           20
help                20
definition          20
cancel              20
instance_manage     20
instance_create     20
instance_connect    20
instance_status     20
message_status      20
message_send        20
message_blast       20
message_schedule    20
contacts            20
chat_view           20
Name: count, dtype: int64

✓ Loaded responses for 32 intents


In [4]:
# ============================================================================
# CELL 4: BUILD VOCABULARY FOR FUZZY MATCHING
# ============================================================================

print("\n" + "=" * 80)
print("📚 BUILDING VOCABULARY FOR FUZZY MATCHING")
print("=" * 80)

# Extract all unique words from training data for typo correction
all_words = set()
for text in df['text'].str.lower():
    all_words.update(re.findall(r'\w+', text))

VOCAB = all_words
print(f"✓ Vocabulary size: {len(VOCAB)} unique words")
print(f"✓ Sample words: {list(VOCAB)[:20]}")

def fuzzy_correct(text: str, cutoff: float = 0.8) -> str:
    """
    Typo correction using difflib (Fuzzy String Matching)
    
    Algorithm: Levenshtein Distance
    - Finds closest matching words from vocabulary
    - Corrects typos while preserving sentence structure
    
    Example: "blst mesage" → "blast message"
    """
    words = re.findall(r'\w+', text.lower())
    corrected = []
    
    for word in words:
        # Find closest match in vocabulary
        matches = get_close_matches(word, VOCAB, n=1, cutoff=cutoff)
        corrected.append(matches[0] if matches else word)
    
    # Reconstruct sentence preserving original punctuation
    result = text
    for orig, corr in zip(words, corrected):
        if orig != corr:
            result = re.sub(rf'\b{orig}\b', corr, result, count=1, flags=re.IGNORECASE)
    
    return result

# Test fuzzy correction
print("\n🧪 Testing Fuzzy Correction:")
test_cases = [
    "blst mesage",
    "chek number",
    "craete instance",
    "shedule mesage"
]

for test in test_cases:
    corrected = fuzzy_correct(test)
    print(f"   '{test}' → '{corrected}'")


📚 BUILDING VOCABULARY FOR FUZZY MATCHING
✓ Vocabulary size: 451 unique words
✓ Sample words: ['simply', 'messages', 'gallery', 'lot', 'customers', 's', 'active', 'files', 'upload', 'catch', 'morning', 'short', 'online', 'preferences', 'afternoon', 'records', 'homepage', 'don', 'where', 'this']

🧪 Testing Fuzzy Correction:
   'blst mesage' → 'blast message'
   'chek number' → 'check number'
   'craete instance' → 'create instance'
   'shedule mesage' → 'schedule message'


In [5]:
# ============================================================================
# CELL 5: LABEL ENCODING
# ============================================================================

print("\n" + "=" * 80)
print("🏷️  ENCODING LABELS")
print("=" * 80)

# Encode intent labels to numeric values
le = LabelEncoder()
df['label'] = le.fit_transform(df['intent'])

num_classes = len(le.classes_)
intent_map = dict(enumerate(le.classes_))

print(f"✓ Number of classes: {num_classes}")
print(f"\n📋 Intent mapping (first 10):")
for idx, intent in list(intent_map.items())[:10]:
    print(f"   {idx}: {intent}")


🏷️  ENCODING LABELS
✓ Number of classes: 33

📋 Intent mapping (first 10):
   0: account
   1: analytics
   2: api
   3: cancel
   4: chat_view
   5: contacts
   6: definition
   7: files
   8: goodbye
   9: gratitude


In [6]:

# ============================================================================
# CELL 6: SENTENCE EMBEDDINGS (WORD2VEC ALTERNATIVE)
# ============================================================================

print("\n" + "=" * 80)
print("🧠 GENERATING SENTENCE EMBEDDINGS")
print("=" * 80)
print("Using: Sentence Transformers (all-MiniLM-L6-v2)")
print("This is a neural embedding model (similar to Word2Vec but sentence-level)")
print("=" * 80)

# Load pre-trained sentence embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')
print("✓ Loaded embedding model")
print(f"✓ Embedding dimension: 384")

# Generate embeddings for all training samples
print("\n📊 Encoding training data...")
sentence_embeddings = embedder.encode(
    df['text'].tolist(), 
    convert_to_tensor=True,
    show_progress_bar=True
).to(device)

print(f"✓ Generated embeddings: {sentence_embeddings.shape}")
print(f"   - Shape: (num_samples, embedding_dim)")
print(f"   - Device: {sentence_embeddings.device}")


🧠 GENERATING SENTENCE EMBEDDINGS
Using: Sentence Transformers (all-MiniLM-L6-v2)
This is a neural embedding model (similar to Word2Vec but sentence-level)
✓ Loaded embedding model
✓ Embedding dimension: 384

📊 Encoding training data...


Batches: 100%|██████████| 21/21 [00:00<00:00, 58.81it/s]

✓ Generated embeddings: torch.Size([657, 384])
   - Shape: (num_samples, embedding_dim)
   - Device: cuda:0


In [7]:

# ============================================================================
# CELL 7: FEEDFORWARD NETWORK
# ============================================================================

print("\n" + "=" * 80)
print("🏗️  FEEDFORWARD CLASSIFIER ARCHITECTURE")
print("=" * 80)

class EmbeddingClassifier(nn.Module):
    """
    Simple Feedforward Neural Network for Sentence Embeddings
    
    Why not CNN?
    - CNNs are for sequential data (words in sentence)
    - We already have holistic sentence embeddings (384-dim vectors)
    - Feedforward network is the right architecture for this!
    
    Architecture:
    1. Input: 384-dim sentence embedding
    2. Hidden Layer 1: 384 → 256 (ReLU + Dropout)
    3. Hidden Layer 2: 256 → 128 (ReLU + Dropout)
    4. Output Layer: 128 → num_classes
    """
    
    def __init__(self, embed_dim=384, num_classes=num_classes):
        super().__init__()
        
        self.network = nn.Sequential(
            # Layer 1
            nn.Linear(embed_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            # Layer 2
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            # Output layer
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        # x shape: (batch_size, 384)
        return self.network(x)  # (batch_size, num_classes)

# Print model architecture
print("\n📐 Model Architecture:")
model = EmbeddingClassifier()
print(model)
print(f"\n✓ Total parameters: {sum(p.numel() for p in model.parameters()):,}")
print("✓ Model defined successfully")


🏗️  FEEDFORWARD CLASSIFIER ARCHITECTURE

📐 Model Architecture:
EmbeddingClassifier(
  (network): Sequential(
    (0): Linear(in_features=384, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=128, out_features=33, bias=True)
  )
)

✓ Total parameters: 135,713
✓ Model defined successfully


In [8]:

# ============================================================================
# CELL 8: TRAIN OR LOAD MODEL
# ============================================================================

print("\n" + "=" * 80)
print("🎯 TRAINING MLP MODEL")
print("=" * 80)

model_path = "chattea.pth"

# Force retrain
import os
if os.path.exists(model_path):
    os.remove(model_path)
    print("⚠️  Deleted old model - retraining from scratch!")

print("\n" + "=" * 80)
print("📚 PREPARING TRAINING DATA")
print("=" * 80)

# Use the pre-computed embeddings from Cell 6
X = sentence_embeddings.to(device)
y = torch.tensor(df['label'].values, dtype=torch.long).to(device)

print(f"✓ X shape: {X.shape}")
print(f"✓ y shape: {y.shape}")

# Train/validation split (stratified)
from sklearn.model_selection import train_test_split

train_idx, val_idx = train_test_split(
    list(range(len(X))),
    test_size=0.2,
    random_state=42,
    stratify=y.cpu()
)

X_train = X[train_idx]
X_val = X[val_idx]
y_train = y[train_idx]
y_val = y[val_idx]

print(f"\n✓ Training samples: {len(X_train)}")
print(f"✓ Validation samples: {len(X_val)}")

print("\n" + "=" * 80)
print("🏋️  TRAINING LOOP (WITH PROPER BATCHING!)")
print("=" * 80)

# Initialize model
model = EmbeddingClassifier().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Lower LR
criterion = nn.CrossEntropyLoss()

# Training parameters
BATCH_SIZE = 32
EPOCHS = 50  # More epochs

# Create mini-batches
def create_batches(X, y, batch_size):
    """Create mini-batches for training"""
    indices = torch.randperm(len(X))
    for i in range(0, len(X), batch_size):
        batch_idx = indices[i:i+batch_size]
        yield X[batch_idx], y[batch_idx]

# Training loop
print("\nEpoch | Train Acc | Train Loss | Val Acc | Val Loss")
print("-" * 65)

best_val_acc = 0

for epoch in range(EPOCHS):
    # ========== TRAINING ==========
    model.train()
    train_loss = 0
    train_correct = 0
    train_total = 0
    
    for batch_X, batch_y in create_batches(X_train, y_train, BATCH_SIZE):
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Track metrics
        train_loss += loss.item()
        train_correct += (outputs.argmax(1) == batch_y).sum().item()
        train_total += len(batch_y)
    
    train_acc = train_correct / train_total
    train_loss = train_loss / (len(X_train) // BATCH_SIZE + 1)
    
    # ========== VALIDATION ==========
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, y_val).item()
        val_acc = (val_outputs.argmax(1) == y_val).float().mean().item()
    
    # Print progress
    if epoch % 5 == 0 or epoch == EPOCHS - 1:
        print(f"{epoch:5d} | {train_acc:9.4f} | {train_loss:10.4f} | {val_acc:7.4f} | {val_loss:8.4f}")
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), model_path)

print("\n" + "=" * 80)
print(f"✓ Training Complete!")
print(f"✓ Best Validation Accuracy: {best_val_acc:.4f} ({best_val_acc*100:.2f}%)")
print(f"✓ Model saved to: {model_path}")
print("=" * 80)

# Load best model
model.load_state_dict(torch.load(model_path))
model.eval()

# Final check on training data
with torch.no_grad():
    final_train_pred = model(X_train).argmax(1)
    final_train_acc = (final_train_pred == y_train).float().mean().item()
    
    final_val_pred = model(X_val).argmax(1)
    final_val_acc = (final_val_pred == y_val).float().mean().item()

print(f"\n📊 Final Performance:")
print(f"   Training Accuracy:   {final_train_acc:.4f} ({final_train_acc*100:.2f}%)")
print(f"   Validation Accuracy: {final_val_acc:.4f} ({final_val_acc*100:.2f}%)")

print("\n✓ Model ready for inference!")


🎯 TRAINING MLP MODEL

📚 PREPARING TRAINING DATA
✓ X shape: torch.Size([657, 384])
✓ y shape: torch.Size([657])

✓ Training samples: 525
✓ Validation samples: 132

🏋️  TRAINING LOOP (WITH PROPER BATCHING!)

Epoch | Train Acc | Train Loss | Val Acc | Val Loss
-----------------------------------------------------------------
    0 |    0.0533 |     3.4865 |  0.1439 |   3.4596
    5 |    0.7067 |     1.6769 |  0.7955 |   1.3960
   10 |    0.9067 |     0.4939 |  0.9091 |   0.5397
   15 |    0.9429 |     0.2574 |  0.8712 |   0.4169
   20 |    0.9733 |     0.1672 |  0.9015 |   0.3774
   25 |    0.9848 |     0.1021 |  0.9091 |   0.3528
   30 |    0.9848 |     0.0891 |  0.8939 |   0.3578
   35 |    0.9829 |     0.0692 |  0.9167 |   0.3336
   40 |    0.9848 |     0.0649 |  0.9091 |   0.3472
   45 |    0.9886 |     0.0446 |  0.9091 |   0.3555
   49 |    0.9829 |     0.0515 |  0.9242 |   0.3709

✓ Training Complete!
✓ Best Validation Accuracy: 0.9242 (92.42%)
✓ Model saved to: chattea.pth

📊 Fina

In [9]:
# ============================================================================
# CELL 9: PHONE NUMBER EXTRACTION (NER)
# ============================================================================

print("\n" + "=" * 80)
print("📱 PHONE NUMBER EXTRACTION (Named Entity Recognition)")
print("=" * 80)

# Regex pattern for Indonesian phone numbers
PHONE_PATTERN = re.compile(
    r'(\b08[1-9]\d{7,12}\b|\b628[1-9]\d{7,12}\b|\b\+628[1-9]\d{7,12}\b)', 
    re.IGNORECASE
)

def extract_phone(text):
    """
    Extract and normalize Indonesian phone numbers
    
    Supports formats:
    - 08123456789 (local)
    - 628123456789 (international without +)
    - +628123456789 (international with +)
    
    Returns normalized format: 08xxxxxxxxxx
    """
    phone_match = PHONE_PATTERN.search(text)
    
    if phone_match:
        num = phone_match.group(0).replace(" ", "").replace("-", "")
        
        if num.startswith("08"):
            return num
        elif num.startswith("628"):
            return "0" + num[2:]
        elif num.startswith("+628"):
            return "0" + num[3:]
    
    return None

# Test phone extraction
print("\n🧪 Testing Phone Extraction:")
test_phones = [
    "check 08123456789",
    "verify 628123456789",
    "validate +628123456789",
    "is 0812-3456-789 valid"
]

for test in test_phones:
    phone = extract_phone(test)
    print(f"   '{test}' → {phone}")


📱 PHONE NUMBER EXTRACTION (Named Entity Recognition)

🧪 Testing Phone Extraction:
   'check 08123456789' → 08123456789
   'verify 628123456789' → 08123456789
   'validate +628123456789' → 08123456789
   'is 0812-3456-789 valid' → None


In [10]:
# ============================================================================
# CELL 10: MAIN CHAT FUNCTION (INFERENCE)
# ============================================================================

print("\n" + "=" * 80)
print("💬 CHAT INFERENCE FUNCTION")
print("=" * 80)

def get_chattea_reply(user_input: str) -> str:
    """
    Main chatbot inference function
    
    Pipeline:
    1. Rule-based filters (greetings, goodbyes)
    2. Phone number extraction (if applicable)
    3. CNN classification with confidence check
    4. Retrieval fallback (if low confidence)
    5. Response generation
    
    Args:
        user_input: User's message
        
    Returns:
        Bot's response
    """
    text = user_input.strip().lower()
    
    # ==================== RULE-BASED FILTERS ====================
    # Quick responses for common greetings
    if any(g in text for g in ["hai", "halo", "hello", "hi", "hey", "pagi", "siang", "malam"]):
        return RESPONSES["greeting"]["en"]
    
    if any(g in text for g in ["bye", "goodbye", "dadah", "sampai jumpa"]):
        return RESPONSES["goodbye"]["en"]
    
    # ==================== PHONE EXTRACTION ====================
    extracted_phone = extract_phone(user_input)
    
    # ==================== EMBEDDING + PREDICTION ====================
    with torch.no_grad():
        # Encode user input
        user_emb = embedder.encode(user_input, convert_to_tensor=True).to(device)
        user_emb = user_emb.unsqueeze(0)  # (1, 384)
        
        # MLP prediction
        logits = model(user_emb)
        probs = logits.softmax(1)
        confidence = probs.max().item()
        intent = intent_map[logits.argmax(1).item()]
        
        # Retrieval fallback (semantic similarity)
        cos_scores = util.cos_sim(user_emb, sentence_embeddings)[0]
        best_match_idx = cos_scores.argmax().item()
        retrieval_intent = df.iloc[best_match_idx]['intent']
        retrieval_score = cos_scores[best_match_idx].item()
        
        # Choose final intent based on confidence
        if confidence > 0.90:
            final_intent = intent
            source = "MLP"
        else:
            final_intent = retrieval_intent
            source = "Retrieval"
    
    # ==================== SPECIAL CASES ====================
    # Phone check with extracted number
    if final_intent == "phone_check" and extracted_phone:
        return f"Checking {extracted_phone}...\nYes, this number is registered and active on WhatsApp!\n\nYou can safely include it in your blast list."
    
    # ==================== RESPONSE GENERATION ====================
    response = RESPONSES.get(final_intent, RESPONSES.get("help", "I'm not sure how to help with that."))
    
    # Handle both dict (bilingual) and string responses
    if isinstance(response, dict):
        return response.get("en", response.get("id", "I'm not sure how to help with that."))
    
    return response

print("✓ Chat function ready!")


💬 CHAT INFERENCE FUNCTION
✓ Chat function ready!


In [11]:

# ============================================================================
# CELL 11: TEST INFERENCE
# ============================================================================

print("\n" + "=" * 80)
print("🧪 TESTING INFERENCE")
print("=" * 80)

test_queries = [
    "hello",
    "what is chattea",
    "how to blast message",
    "check 08123456789",
    "create instance",
    "schedule message",
    "thanks",
    "goodbye"
]

print("\nRunning test queries:\n")
for query in test_queries:
    print(f"👤 User: {query}")
    response = get_chattea_reply(query)
    print(f"🤖 Bot: {response[:100]}{'...' if len(response) > 100 else ''}")
    print("-" * 80)



🧪 TESTING INFERENCE

Running test queries:

👤 User: hello
🤖 Bot: Hello! 👋 Welcome to Chattea.

I'm here to help you navigate features like sending messages, managing...
--------------------------------------------------------------------------------
👤 User: what is chattea
🤖 Bot: Chattea is a WhatsApp marketing automation platform designed for businesses.

Key features:
• Send m...
--------------------------------------------------------------------------------
👤 User: how to blast message
🤖 Bot: To send a blast (mass message):

1. Go to **Blast Message** → **New Blast**
2. Select contacts, grou...
--------------------------------------------------------------------------------
👤 User: check 08123456789
🤖 Bot: Checking 08123456789...
Yes, this number is registered and active on WhatsApp!

You can safely inclu...
--------------------------------------------------------------------------------
👤 User: create instance
🤖 Bot: To create a new WhatsApp instance:

1. Open the **Instances** t

In [12]:

# ============================================================================
# CELL 13: MODEL EVALUATION
# ============================================================================

print("\n" + "=" * 80)
print("📊 MODEL EVALUATION")
print("=" * 80)

# Evaluate on validation set
with torch.no_grad():
    # Get embeddings
    X_all = embedder.encode(df['text'].tolist(), convert_to_tensor=True).to(device)
    y_all = torch.tensor(df['label'].values, dtype=torch.long).to(device)
    
    # Split
    train_idx, val_idx = train_test_split(
        torch.arange(len(X_all)),
        test_size=0.2,
        random_state=42,
        stratify=y_all.cpu()
    )
    
    X_val = X_all[val_idx].to(device)
    y_val = y_all[val_idx].to(device)
    
    # Predict
    val_outputs = model(X_val)
    val_preds = val_outputs.argmax(1)
    
    # Accuracy
    val_acc = (val_preds == y_val).float().mean().item()
    
    print(f"✓ Validation Accuracy: {val_acc:.4f} ({val_acc*100:.2f}%)")
    
    # Per-class accuracy
    print("\n📋 Per-Intent Performance:")
    for intent_id in range(num_classes):
        intent_name = intent_map[intent_id]
        mask = y_val == intent_id
        if mask.sum() > 0:
            intent_acc = (val_preds[mask] == y_val[mask]).float().mean().item()
            count = mask.sum().item()
            print(f"   {intent_name:30s}: {intent_acc:.3f} ({count:2d} samples)")

# Test on ALL training data (should be near perfect)
with torch.no_grad():
    all_outputs = model(sentence_embeddings.to(device))
    all_preds = all_outputs.argmax(1)
    all_labels = torch.tensor(df['label'].values, dtype=torch.long).to(device)
    
    train_acc = (all_preds == all_labels).float().mean().item()
    print(f"Accuracy on FULL training set: {train_acc:.4f}")

print("\n" + "=" * 80)
print("✅ NOTEBOOK COMPLETE!")
print("=" * 80)
print("\nYour model is ready to use!")

# ============================================================================
# FRESH START - DELETE EVERYTHING AND RETRAIN
# ============================================================================

import os

# 1. Delete saved model
if os.path.exists("chattea.pth"):
    os.remove("chattea.pth")
    print("✓ Deleted old model")

# 2. Clear GPU cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("✓ Cleared GPU cache")

# 3. Restart notebook kernel (Kernel → Restart & Run All)
print("\n⚠️  NOW RESTART KERNEL AND RUN ALL CELLS FROM TOP!")


📊 MODEL EVALUATION
✓ Validation Accuracy: 0.9242 (92.42%)

📋 Per-Intent Performance:
   account                       : 0.750 ( 4 samples)
   analytics                     : 1.000 ( 4 samples)
   api                           : 1.000 ( 4 samples)
   cancel                        : 1.000 ( 4 samples)
   chat_view                     : 1.000 ( 4 samples)
   contacts                      : 1.000 ( 4 samples)
   definition                    : 1.000 ( 4 samples)
   files                         : 0.750 ( 4 samples)
   goodbye                       : 0.750 ( 4 samples)
   gratitude                     : 1.000 ( 4 samples)
   greeting                      : 1.000 ( 4 samples)
   groups                        : 1.000 ( 4 samples)
   help                          : 1.000 ( 4 samples)
   instance_connect              : 0.750 ( 4 samples)
   instance_create               : 1.000 ( 4 samples)
   instance_manage               : 1.000 ( 4 samples)
   instance_status               : 1.000 ( 4 sampl